In [57]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [73]:
# Read the csv data file as data_df
data_df = pd.read_csv("data/Billionaires Statistics Dataset.csv")
data_df.head()

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,organization,selfMade,status,gender,birthDate,lastName,firstName,title,date,state,residenceStateRegion,birthYear,birthMonth,birthDay,cpi_country,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,LVMH Moët Hennessy Louis Vuitton,False,U,M,3/5/1949 0:00,Arnault,Bernard,Chairman and CEO,4/4/2023 5:01,NaN,NaN,1949.0,3.0,5.0,110.05,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,Tesla,True,D,M,6/28/1971 0:00,Musk,Elon,CEO,4/4/2023 5:01,Texas,South,1971.0,6.0,28.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,Amazon,True,D,M,1/12/1964 0:00,Bezos,Jeff,Chairman and Founder,4/4/2023 5:01,Washington,West,1964.0,1.0,12.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,Oracle,True,U,M,8/17/1944 0:00,Ellison,Larry,CTO and Founder,4/4/2023 5:01,Hawaii,West,1944.0,8.0,17.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,Berkshire Hathaway Inc. (Cl A),True,D,M,8/30/1930 0:00,Buffett,Warren,CEO,4/4/2023 5:01,Nebraska,Midwest,1930.0,8.0,30.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


In [78]:
# Check out the data types
data_df.dtypes

# Convert gdp_country to float

# Convert finalWorth unit to billion
data_df['finalWorth'] = data_df['finalWorth']/1000

In [100]:
# Checking rows that contain nan value
#data_df[data_df.isna().any(axis=1)]
data_df.isnull().sum()

rank                                             0
finalWorth                                       0
category                                         0
personName                                       0
age                                             65
country                                         38
city                                            72
source                                           0
industries                                       0
countryOfCitizenship                             0
organization                                  2315
selfMade                                         0
status                                           0
gender                                           0
birthDate                                       76
lastName                                         0
firstName                                        3
title                                         2301
date                                             0
state                          

In [87]:
data_df.shape

(2640, 35)

In [117]:
# Cleaning out the 'State' and 'Residence of State' which are not needed
full_df = data_df[['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
    'source', 'industries', 'countryOfCitizenship','selfMade', 'status', 'gender', 'lastName', 'firstName',
       'cpi_country', 'cpi_change_country',
       'gdp_country', 'gross_tertiary_education_enrollment',
       'gross_primary_education_enrollment_country', 'life_expectancy_country',
       'tax_revenue_country_country', 'total_tax_rate_country',
       'population_country', 'latitude_country', 'longitude_country']]
full_df.isnull().sum()

rank                                            0
finalWorth                                      0
category                                        0
personName                                      0
age                                            65
country                                        38
source                                          0
industries                                      0
countryOfCitizenship                            0
selfMade                                        0
status                                          0
gender                                          0
lastName                                        0
firstName                                       3
cpi_country                                   184
cpi_change_country                            184
gdp_country                                   164
gross_tertiary_education_enrollment           182
gross_primary_education_enrollment_country    181
life_expectancy_country                       182


In [122]:
df = full_df[full_df['country'].isnull()]
df

,rank,finalWorth,category,personName,age,country,source,industries,countryOfCitizenship,selfMade,status,gender,lastName,firstName,cpi_country,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
108,108,15.8,Fashion & Retail,Beate Heister,NaN,NaN,Supermarkets,Fashion & Retail,Germany,False,Split Family Fortune,F,Heister,Beate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,418,6.1,Telecom,Sunil Mittal,65.0,NaN,Telecom,Telecom,India,True,Split Family Fortune,M,Mittal,Sunil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,636,4.3,Real Estate,Ian Livingstone,60.0,NaN,Real estate,Real Estate,United Kingdom,True,Split Family Fortune,M,Livingstone,Ian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,636,4.3,Real Estate,Richard Livingstone,58.0,NaN,Real estate,Real Estate,United Kingdom,True,Split Family Fortune,M,Livingstone,Richard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
904,905,3.2,Food & Beverage,Karen Virginia Beckmann Legoretta,53.0,NaN,Tequila,Food & Beverage,Mexico,False,U,F,Beckmann Legoretta,Karen Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,982,3.0,Automotive,Catheline Perier D'Ieteren,78.0,NaN,"Auto parts, distribution",Automotive,Belgium,False,N,F,D'Ieteren,Catheline Perier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141,1104,2.7,Food & Beverage,Timm Oberwelland,NaN,NaN,Candy,Food & Beverage,Germany,False,Split Family Fortune,M,Oberwelland,Timm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1147,1104,2.7,Metals & Mining,Gianfelice Rocca,75.0,NaN,Pipe Manufacturing,Metals & Mining,Italy,False,Split Family Fortune,M,Rocca,Gianfelice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1148,1104,2.7,Metals & Mining,Paolo Rocca,70.0,NaN,Pipe Manufacturing,Metals & Mining,Italy,False,Split Family Fortune,M,Rocca,Paolo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1152,1104,2.7,Media & Entertainment,Christiane Schoeller,NaN,NaN,Publishing,Media & Entertainment,Germany,False,U,F,Schoeller,Christiane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
df = full_df[full_df['cpi_country'].isna()]
df

,rank,finalWorth,category,personName,age,country,source,industries,countryOfCitizenship,selfMade,status,gender,lastName,firstName,cpi_country,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
32,33,38.0,Diversified,Li Ka-shing,94.0,Hong Kong,Diversified,Diversified,Hong Kong,True,U,M,Li,Ka-shing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,47,29.5,Real Estate,Lee Shau Kee,95.0,Hong Kong,Real estate,Real Estate,Hong Kong,True,D,M,Lee,Shau Kee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,86,18.9,Diversified,Eyal Ofer,72.0,Monaco,"Real estate, shipping",Diversified,Israel,False,U,M,Ofer,Eyal,NaN,NaN,"$7,184,844,193",NaN,NaN,NaN,NaN,NaN,38964.0,43.738418,7.424616
108,108,15.8,Fashion & Retail,Beate Heister,NaN,NaN,Supermarkets,Fashion & Retail,Germany,False,Split Family Fortune,F,Heister,Beate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,108,15.8,Real Estate,Peter Woo,76.0,Hong Kong,Real estate,Real Estate,Hong Kong,False,D,M,Woo,Peter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,122,14.4,Metals & Mining,Alisher Usmanov,69.0,Uzbekistan,"Steel, telecom, investments",Metals & Mining,Russia,True,U,M,Usmanov,Alisher,NaN,NaN,"$57,921,286,440",10.1,104.2,71.6,14.8,31.6,33580650.0,41.377491,64.585262
125,126,14.1,Real Estate,Kwong Siu-hing,93.0,Hong Kong,Real estate,Real Estate,Hong Kong,False,U,F,Kwong,Siu-hing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,128,13.9,Gambling & Casinos,Lui Che Woo,94.0,Hong Kong,Casinos/hotels,Gambling & Casinos,Hong Kong,True,U,M,Lui,Che Woo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,133,13.3,Real Estate,Joseph Lau,71.0,Hong Kong,Real estate,Real Estate,Hong Kong,True,D,M,Lau,Joseph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,146,11.8,Diversified,Robert Kuok,99.0,Hong Kong,"Palm oil, shipping, property",Diversified,Malaysia,True,U,M,Kuok,Robert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# General Informations About Billionaires Distributions

In [84]:
# Gender demographic of billionaires

# Gender proportion pie chart

# Gender with rank and wealth


In [ ]:
# Age distribution of billionaires by gender

# histogram

# boxplot to see the distribution within gender


In [ ]:
# Billionaires by industries

# Histogram

In [ ]:
# Billionaires by countries

# histogram

In [ ]:
# Geographical distribution

# Top 10 countries with most billionaires

# Use API geoapify to map the countries of billionaires


In [ ]:
# Selfmade billionaire vs. Inherited billionaire


# Correlation Analysis of Billionaires

In [ ]:
# Billionaire vs. Countries GDP


In [ ]:
# Billionaire vs. Countries Life Expectancies


In [ ]:
# Billionaires vs. Countries Population


In [ ]:
# Billionaire vs. Countries Tax Revenue


In [ ]:
# Billionaire vs. Countries CPI


In [ ]:
# Billionaire vs. Education


# Bonus: The chances of you becoming a billionaire

In [85]:
# We are only interested in selfmade billionaire for this part
# Y = billionaire/countries population %
# X = gender + country + age + industries

